# Silver Layer: Data Cleansing and Deduplication
**Purpose:**
- Remove duplicate trades
- Apply data quality rules
- Add business logic (e.g. VWAP calculation)

**Exam Topics:**
- Watermarking for stateful operations
- Deduplications with `dropDuplications`
- MERGE operations in Delta Lake

## 1. Configuration

In [0]:
# Imports
from pyspark.sql.functions import (
    col, sum, avg, count, window, expr,
    when, lit, round as spark_round
)
from delta.tables import DeltaTable

In [0]:
# Paths
BRONZE_TABLE = "finance_lakehouse.bronze.market_trades_raw"
SILVER_TABLE = "finance_lakehouse.silver.market_trades_clean"
CHECKPOINT_PATH = "/tmp/checkpoints/silver_cleansing"

## 2. Read from Bronze (Streaming)

In [0]:
bronze_stream = (
    spark.readStream
    .format("delta")
    .table(BRONZE_TABLE)
)

## 3. Data Quality Transformations

In [0]:
cleaned_stream = (
    bronze_stream
    # Data Quality Filters
    .filter(col("price") > 0)
    .filter(col("qty") > 0)
    .filter(col("symbol").isNotNull())

    # Deduplication with watermarking
    .withWatermark("event_datetime", "10 minutes")
    .dropDuplicates(["exchange", "symbol", "id"])

    # Add derived columns
    .withColumn("trade_value_usd", spark_round(col("price") * col("qty"), 2))
    .withColumn("is_large_trade", when(col("qty") > 1.0, True).otherwise(False))
)

## 4. Write to Silver (Delta Lake)

In [0]:
silver_query = (
  cleaned_stream.writeStream
  .format("delta")
  .outputMode("append")
  .option("checkpointLocation", CHECKPOINT_PATH)
  .trigger(availableNow=True)
  .table(SILVER_TABLE)
)

In [0]:
for stream in spark.streams.active:
  print(f"Stream ID: {stream.id}")
  print(f"Run ID: {stream.runId}")
  print(f"Status: {stream.status}")
  print(f"Recent Progress:\n{stream.lastProgress}")
  print("-" * 80)

## 5. Verify Silver Data Quality

In [0]:
%sql
-- Data quality check: No null symbols
SELECT COUNT(*) as null_symbols
FROM finance_lakehouse.silver.market_trades_clean
WHERE symbol IS NULL;

In [0]:
%sql
-- Check for duplicates (should be 0)
SELECT exchange, symbol, id, COUNT(*) as dup_count
FROM finance_lakehouse.silver.market_trades_clean
GROUP BY exchange, symbol, id
HAVING COUNT(*) > 1;

In [0]:
%sql
SELECT
  side,
  COUNT(*) AS trade_counts,
  SUM(CASE WHEN is_large_trade THEN 1 ELSE 0 END) as large_trade_count,
  AVG(trade_value_usd) as avg_trade_value
FROM finance_lakehouse.silver.market_trades_clean
GROUP BY side;

## Stop Stream


In [0]:
# Stop the stream
silver_query.stop()

In [0]:
# Verify that the stream has been turned off
for stream in spark.streams.active:
  print(f"Stream ID: {stream.id}")
  print(f"Run ID: {stream.runId}")
  print(f"Status: {stream.status}")
  print(f"Recent Progress:\n{stream.lastProgress}")
  print("-" * 80)